In [1]:
# !pip install trectools
import pandas as pd
import trectools as trec

In [2]:
results_df = pd.read_csv("./MQ2007-agg/result_agg_functions.csv", sep="\t")
results_df.head(5)

,qid,docid,relevance,ranking_1,ranking_2,ranking_3,ranking_4,ranking_5,ranking_6,ranking_7,...,ranking_15_rank,ranking_16_rank,ranking_17_rank,ranking_18_rank,ranking_19_rank,ranking_20_rank,ranking_21_rank,rrf,squared_rrf,mrrf
0,10,GX000-00-0000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
1,10,GX000-24-12369390,1,107,107,85,0,113,110,110,...,3,8,0,7,6,14,6,0.246088,0.435389,2.837641
2,10,GX000-62-7863450,1,0,0,0,1,8,14,14,...,29,4,0,0,7,0,0,0.160114,0.228318,1.719032
3,10,GX016-48-5543459,1,0,0,111,4,1,3,3,...,26,10,3,1,18,7,1,0.241809,2.211949,2.620832
4,10,GX037-87-3082362,0,160,160,117,57,102,209,209,...,17,7,0,0,13,22,17,0.240359,0.369523,2.714266


In [3]:
qrel_df = results_df[["qid", "docid", "relevance"]].copy()
qrel_df['rank'] = qrel_df.groupby('qid')['relevance'].rank(method='dense', ascending=False).astype(int)
qrel_df['q0']  = 'Q0'
qrel_df['rel'] = qrel_df['relevance']
qrel_df['query'] = qrel_df['qid']
qrel_df = qrel_df[["qid","query", "q0", "docid", "rel"]]
print(qrel_df.head(5))
trec_qrel = trec.TrecQrel()
trec_qrel.qrels_data = qrel_df.copy()

   qid  query  q0              docid  rel
0   10     10  Q0   GX000-00-0000000    0
1   10     10  Q0  GX000-24-12369390    1
2   10     10  Q0   GX000-62-7863450    1
3   10     10  Q0   GX016-48-5543459    1
4   10     10  Q0   GX037-87-3082362    0


In [4]:
# Evaluate RRF
run_df_rrf = results_df[["qid", "docid", "rrf"]].copy()
run_df_rrf['rank'] = run_df_rrf.groupby('qid')['rrf'].rank(method='dense', ascending=False).astype(int)
run_df_rrf['q0']  = 'Q0'
run_df_rrf['score'] = run_df_rrf['rrf']
run_df_rrf['system'] = 'rrf'
run_df_rrf['query'] = run_df_rrf['qid']
run_df_rrf = run_df_rrf[["qid","query","q0", "docid","rank" ,"score", "system"]]
# print(run_df_rrf.head(5))

# RRF Evaluation
run_rrf = trec.TrecRun()
run_rrf.run_data = run_df_rrf.copy()
rrf_trec_eval = trec.TrecEval(run_rrf, trec_qrel)
print(f"rrd:{rrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"ndcg@{i}:{rrf_trec_eval.get_ndcg(depth=i, trec_eval=False)}")
    print(f"precision@{i}:{rrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"map@{i}:{rrf_trec_eval.get_map(depth=i, trec_eval=False)}")
    


rrd:17991
ndcg@1:0.10874704491725769
precision@1:0.13356973995271867
map@1:0.012805480075131045
ndcg@2:0.14575043341856744
precision@2:0.18942080378250592
map@2:0.025561402466285615
ndcg@3:0.16778073830998738
precision@3:0.21591804570527973
map@3:0.03701029500787046
ndcg@4:0.18196898988482083
precision@4:0.2285756501182033
map@4:0.04714805302650744
ndcg@5:0.19740817167531383
precision@5:0.24125295508274233
map@5:0.05792235932448375
ndcg@6:0.21055270617943697
precision@6:0.2489164696611505
map@6:0.06804788560651918
ndcg@7:0.22085057002034647
precision@7:0.25329280648429586
map@7:0.07745914278687352
ndcg@8:0.23069451324307744
precision@8:0.2563534278959811
map@8:0.0861445835341878
ndcg@9:0.24003661472162094
precision@9:0.25925925925925924
map@9:0.09519528367192441
ndcg@10:0.24893307424044056
precision@10:0.2617612293144208
map@10:0.10378600017626864


In [5]:
# Evaluate SRRF
run_df_srrf = results_df[["qid", "docid", "squared_rrf"]].copy()
run_df_srrf['rank'] = run_df_srrf.groupby('qid')['squared_rrf'].rank(method='dense', ascending=False).astype(int)
run_df_srrf['q0']  = 'Q0'
run_df_srrf['score'] = run_df_srrf['squared_rrf']
run_df_srrf['system'] = 'srrf'
run_df_srrf['query'] = run_df_srrf['qid']
run_df_srrf = run_df_srrf[["qid","query","q0", "docid","rank" ,"score", "system"]]
# print(run_df_srrf.head(5))

# SRRF Evaluation
run_srrf = trec.TrecRun()
run_srrf.run_data = run_df_srrf.copy()
srrf_trec_eval = trec.TrecEval(run_srrf, trec_qrel)
print(f"rrd:{srrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    # print(f"ndcg@{i}:{srrf_trec_eval.get_ndcg(depth=i)}")
    print(f"ndcg@{i}:{srrf_trec_eval.get_ndcg(depth=i, trec_eval=False)}")
    print(f"precision@{i}:{srrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"map@{i}:{srrf_trec_eval.get_map(depth=i, trec_eval=False)}")
    

rrd:17991
ndcg@1:0.10874704491725769
precision@1:0.13356973995271867
map@1:0.012805480075131045
ndcg@2:0.14575043341856744
precision@2:0.18942080378250592
map@2:0.025561402466285615
ndcg@3:0.16778073830998738
precision@3:0.21591804570527973
map@3:0.03701029500787046
ndcg@4:0.18196898988482083
precision@4:0.2285756501182033
map@4:0.04714805302650744
ndcg@5:0.19740817167531383
precision@5:0.24125295508274233
map@5:0.05792235932448375
ndcg@6:0.21055270617943697
precision@6:0.2489164696611505
map@6:0.06804788560651918
ndcg@7:0.22085057002034647
precision@7:0.25329280648429586
map@7:0.07745914278687352
ndcg@8:0.23069451324307744
precision@8:0.2563534278959811
map@8:0.0861445835341878
ndcg@9:0.24003661472162094
precision@9:0.25925925925925924
map@9:0.09519528367192441
ndcg@10:0.24893307424044056
precision@10:0.2617612293144208
map@10:0.10378600017626864


In [6]:
# Evaluate MRRF
run_df_mrrf = results_df[["qid", "docid", "mrrf"]].copy()
run_df_mrrf['rank'] = run_df_mrrf.groupby('qid')['mrrf'].rank(method='dense', ascending=False).astype(int)
run_df_mrrf['q0']  = 'Q0'
run_df_mrrf['score'] = run_df_mrrf['mrrf']
run_df_mrrf['system'] = 'mrrf'
run_df_mrrf['query'] = run_df_mrrf['qid']
run_df_mrrf = run_df_mrrf[["qid","query","q0", "docid","rank", "score", "system"]]
# print(run_df_mrrf.head(5))

# MRRF Evaluation
run_mrrf = trec.TrecRun()
run_mrrf.run_data = run_df_mrrf.copy()
mrrf_trec_eval = trec.TrecEval(run_mrrf, trec_qrel)
print(f"rrd:{mrrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"ndcg@{i}:{mrrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"precision@{i}:{mrrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"map@{i}:{mrrf_trec_eval.get_map(depth=i, trec_eval=False)}")
    

rrd:17991
ndcg@1:0.13356973995271867
precision@1:0.13356973995271867
map@1:0.012805480075131045
ndcg@2:0.18942080378250592
precision@2:0.18942080378250592
map@2:0.025561402466285615
ndcg@3:0.21591804570527973
precision@3:0.21591804570527973
map@3:0.03701029500787046
ndcg@4:0.2285756501182033
precision@4:0.2285756501182033
map@4:0.04714805302650744
ndcg@5:0.24125295508274233
precision@5:0.24125295508274233
map@5:0.05792235932448375
ndcg@6:0.2489164696611505
precision@6:0.2489164696611505
map@6:0.06804788560651918
ndcg@7:0.25329280648429586
precision@7:0.25329280648429586
map@7:0.07745914278687352
ndcg@8:0.2563534278959811
precision@8:0.2563534278959811
map@8:0.0861445835341878
ndcg@9:0.25925925925925924
precision@9:0.25925925925925924
map@9:0.09519528367192441
ndcg@10:0.2617612293144208
precision@10:0.2617612293144208
map@10:0.10378600017626864
